# 子词嵌入


## fastText模型



## 字节对编码（Byte Pair Encoding）


In [1]:
import collections

symbols = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
           'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
           '_', '[UNK]']

因为我们不考虑跨越词边界的符号对，所以我们只需要一个字典`raw_token_freqs`将词映射到数据集中的频率（出现次数）。注意，特殊符号`'_'`被附加到每个词的尾部，以便我们可以容易地从输出符号序列（例如，“a_all er_man”）恢复单词序列（例如，“a_all er_man”）。由于我们仅从单个字符和特殊符号的词开始合并处理，所以在每个词（词典`token_freqs`的键）内的每对连续字符之间插入空格。换句话说，空格是词中符号之间的分隔符。


**步骤分解（以 'fast_' 为例）：**
**list(token) ：将字符串拆分为 字符列表**
```Python
list('fast_')  # → ['f', 'a', 's', 't', '_']
' '.join(...) ：用 空格连接字符
```
```Python
' '.join(['f', 'a', 's', 't', '_'])  # → 'f a s t _'
```
**转换结果**
```Python
token_freqs = {
    'f a s t _': 4,      # 由 'fast_' 转换而来
    'f a s t e r _': 3,  # 由 'faster_' 转换而来
    't a l l _': 5,      # 由 'tall_' 转换而来
    't a l l e r _': 4   # 由 'taller_' 转换而来
}
```
**为什么这样做？**<br>
这种转换通常用于子词级别的模型（如Byte-Pair Encoding, BPE）或字符级语言模型：

1. 用途1：字符级建模
将每个单词视为字符序列，模型学习字符间的模式：<br>
fast_ → f a s t _<br>
模型可以捕捉 "a s t" 这个后缀模式<br>

2. 用途2：子词单元（Subword Units）<br>
在BPE或WordPiece分词中，将单词拆分为更小的单元：<br>
f a s t → 可能合并为 fa st<br>
处理未登录词（OOV）：即使没见过 "fastest"，也能用字符级信息推断<br>

3. 用途3：形态学分析<br>
帮助模型识别词根、前缀、后缀：<br>
fast_ vs faster_：共享 "fast" 部分<br>
er_ 后缀表明比较级

In [2]:
raw_token_freqs = {'fast_': 4, 'faster_': 3, 'tall_': 5, 'taller_': 4}
token_freqs = {}
for token, freq in raw_token_freqs.items():
    token_freqs[' '.join(list(token))] = raw_token_freqs[token]
token_freqs

{'f a s t _': 4, 'f a s t e r _': 3, 't a l l _': 5, 't a l l e r _': 4}

我们定义以下`get_max_freq_pair`函数，其返回词内最频繁的连续符号对，其中词来自输入词典`token_freqs`的键。


| 词项       | 频率 | 字符对        | 累计频率        |
| -------- | -- | ---------- | ----------- |
| fast\_   | 4  | ('f','a')  | +4 → 4      |
|          |    | ('a','s')  | +4 → 4      |
|          |    | ('s','t')  | +4 → 4      |
|          |    | ('t','\_') | +4 → 4      |
| faster\_ | 3  | ('f','a')  | +3 → **7**  |
|          |    | ('a','s')  | +3 → **7**  |
|          |    | ('s','t')  | +3 → **7**  |
|          |    | ('t','e')  | +3 → 3      |
|          |    | ('e','r')  | +3 → 3      |
|          |    | ('r','\_') | +3 → 3      |
| tall\_   | 5  | ('t','a')  | +5 → 9      |
|          |    | ('a','l')  | +5 → 5      |
|          |    | ('l','l')  | +5 → 5      |
|          |    | ('l','\_') | +5 → 5      |
| taller\_ | 4  | ('t','a')  | +4 → **13** |
|          |    | ('a','l')  | +4 → **9**  |
|          |    | ('l','l')  | +4 → **9**  |
|          |    | ('l','e')  | +4 → 4      |
|          |    | ('e','r')  | +4 → **7**  |
|          |    | ('r','\_') | +4 → **7**  |


In [3]:
def get_max_freq_pair(token_freqs):
    # 1. 创建字符对计数器：用途：统计每个字符对的出现频率
    # defaultdict(int)：当访问不存在的键时，自动初始化为0
    pairs = collections.defaultdict(int)
    # 2. 遍历所有词项
    # token.split()：将字符序列拆分为列表（如'f a s t _'→['f','a','s','t','_']）
    for token, freq in token_freqs.items():
        symbols = token.split()
        '''
        3. 统计相邻字符对
        有效对数=字符数-1
        滑动窗口：遍历所有相邻字符对
        (symbols[i],symbols[i+1])：构成字符对元组（如('f','a')）
        +=freq：将该词的出现次数累加到对应字符对
        '''
        for i in range(len(symbols) - 1):
            # “pairs”的键是两个连续符号的元组
            pairs[symbols[i], symbols[i + 1]] += freq
    # 4. 返回最高频字符对
    # pairs.get：获取字符对的频率值；max(...,key=...)：返回频率最高的字符对
    return max(pairs, key=pairs.get)  # 具有最大值的“pairs”键

**输入**
```Python
max_freq_pair = ('t', 'a')
token_freqs = {
    't a l l _': 5,        # tall_
    't a l l e r _': 4,    # taller_
    'f a s t _': 4,        # fast_
}
symbols = ['a', 'b', 'c', ...]  # 已有符号
```
**执行过程**

**Step 1: 创建新符号**
```Python
symbols.append(''.join(('t', 'a')))  # → symbols 增加 'ta'
```
**Step 2: 合并词项**
```Python
# 处理 't a l l _'
new_token = 't a l l _'.replace('t a', 'ta')  # → 'ta l l _'

# 处理 't a l l e r _'
new_token = 't a l l e r _'.replace('t a', 'ta')  # → 'ta l l e r _'

# 处理 'f a s t _'
new_token = 'f a s t _'.replace('t a', 'ta')  # 无 't a'，保持不变
```
**Step 3: 构建新字典**
```Python
new_token_freqs = {
    'ta l l _': 5,        # 原为 't a l l _'
    'ta l l e r _': 4,    # 原为 't a l l e r _'
    'f a s t _': 4,       # 未变化
}
```
**为什么这样设计？**
1. 空格分隔的重要性<br>
字符序列用空格分隔字符（如 't a l l _'），确保只合并相邻字符：
```Python
# 正确：仅合并相邻的 't a'
't a l l _'.replace('t a', 'ta')  # → 'ta l l _'

# 错误：如果用无空格字符串，会错误合并
'tall_'.replace('ta', 'ta')  # 会匹配到 'ta'，但无法保证相邻性
```
2. 保持频率不变<br>
合并操作不改变词频，只改变词形：
```Python
new_token_freqs[new_token] = token_freqs[token]  # 直接复制频率
```
3. 全局替换<br>
replace()会替换所有出现位置：
```Python
't a l l e r _'.replace('t a', 'ta')  # 即使出现多次也会被全部替换
```
**在BPE算法中的位置**

**这是 BPE训练循环的核心步骤：**
```Python
while len(symbols) < vocab_size:
    # 1. 统计字符对频率
    pair = get_max_freq_pair(token_freqs)
    
    # 2. 合并最高频对
    token_freqs = merge_symbols(pair, token_freqs, symbols)
    
    # 3. 重复直到达到目标词汇量
```
**每次迭代：**
- 词汇表新增1个符号（如 'ta'）
- 词频字典更新为合并后形式
- 模型能力增强（能表示更长的子词单元）

In [4]:
def merge_symbols(max_freq_pair, token_freqs, symbols):
    # 1. 创建新符号并加入词汇表：将新符号（如 'ta'）添加到BPE词汇表
    # ''.join(max_freq_pair)：将字符对元组转为字符串（如('t','a')→'ta'）
    symbols.append(''.join(max_freq_pair))
    # 2. 初始化新词频字典：存储合并后的词项及其频率
    new_token_freqs = dict()
    '''
    3. 遍历所有词项进行合并
    ' '.join(max_freq_pair)：将字符对转为带空格的字符串（如't a'）
        token.replace(...)：全局替换所有出现的该字符对
            将't a'→'ta'
            仅替换连续相邻的字符对
    '''
    for token, freq in token_freqs.items():
        new_token = token.replace(' '.join(max_freq_pair),
                                  ''.join(max_freq_pair))
        new_token_freqs[new_token] = token_freqs[token]
    return new_token_freqs

| 轮次      | 最高频对            | 新符号       | 合并后的词项示例                       | 词汇表变化    |
| ------- | --------------- | --------- | ------------------------------ | -------- |
| **#1**  | `('t', 'a')`    | `'ta'`    | `'ta l l _'`, `'ta l l e r _'` | `+ta`    |
| **#2**  | `('l', 'l')`    | `'ll'`    | `'ta ll _'`, `'ta ll e r _'`   | `+ll`    |
| **#3**  | `('ta', 'll')`  | `'tall'`  | `'tall _'`, `'tall e r _'`     | `+tall`  |
| **#4**  | `('e', 'r')`    | `'er'`    | `'tall er _'`, `'fast er _'`   | `+er`    |
| **#5**  | `('fast', '_')` | `'fast_'` | ...                            | `+fast_` |
| **#6**  | `('tall', '_')` | `'tall_'` | ...                            | `+tall_` |
| ...     | ...             | ...       | ...                            | ...      |
| **#10** | `('er', '_')`   | `'er_'`   | ...                            | `+er_`   |


In [5]:
'''
1. 设定合并次数
含义：执行10轮字符对合并
结果：词汇表将新增10个子词单元（如'ta','ll','er'等）
选择依据：根据目标词汇量或计算资源决定
'''
num_merges = 10 # 设定合并次数
for i in range(num_merges):
    '''
    2. 找出最高频字符对
    调用：统计当前所有相邻字符对的频率
    返回：频率最高的字符对元组（如('t','a')）
    '''
    max_freq_pair = get_max_freq_pair(token_freqs) # 找出当前最高频字符对
    '''
    3. 合并字符对并更新数据
    作用：将所有词项中的该字符对合并为新符号
    更新：token_freqs变为合并后的新字典
    副作用：symbols列表追加新符号
    '''
    token_freqs = merge_symbols(max_freq_pair, token_freqs, symbols) # 合并该字符对
    '''
    4. 打印合并进度
    输出示例：合并# 1:('t','a')
    调试用途：观察BPE学习过程
    '''
    print(f'合并# {i+1}:',max_freq_pair) # 打印进度

合并# 1: ('t', 'a')
合并# 2: ('ta', 'l')
合并# 3: ('tal', 'l')
合并# 4: ('f', 'a')
合并# 5: ('fa', 's')
合并# 6: ('fas', 't')
合并# 7: ('e', 'r')
合并# 8: ('er', '_')
合并# 9: ('tall', '_')
合并# 10: ('fast', '_')


在字节对编码的10次迭代之后，我们可以看到列表`symbols`现在又包含10个从其他符号迭代合并而来的符号。


In [6]:
print(symbols)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '_', '[UNK]', 'ta', 'tal', 'tall', 'fa', 'fas', 'fast', 'er', 'er_', 'tall_', 'fast_']


对于在词典`raw_token_freqs`的键中指定的同一数据集，作为字节对编码算法的结果，数据集中的每个词现在被子词“fast_”“fast”“er_”“tall_”和“tall”分割。例如，单词“fast er_”和“tall er_”分别被分割为“fast er_”和“tall er_”。


| 代码片段                 | 作用     | 示例输出                                          |
| -------------------- | ------ | --------------------------------------------- |
| `token_freqs.keys()` | 获取所有词项 | `dict_keys(['t a l l _', ...])`               |
| `list(...)`          | 转为列表   | `['t a l l _', 't a l l e r _', 'f a s t _']` |
| `print(...)`         | 打印查看   | 显示在控制台，用于监控                                   |


In [7]:
print(list(token_freqs.keys()))

['fast_', 'fast er_', 'tall_', 'tall er_']


请注意，字节对编码的结果取决于正在使用的数据集。我们还可以使用从一个数据集学习的子词来切分另一个数据集的单词。作为一种贪心方法，下面的`segment_BPE`函数尝试将单词从输入参数`symbols`分成可能最长的子词。


**算法流程：**
1. 从整词开始：token[0:len(token)]（如 'tallest_'）
2. 是否在词汇表：
- 是：找到最长子词！添加到结果，移动指针到剩余部分
- 否：将 end 减1，尝试更短的子串
3. 重复：直到匹配或无法继续
**示例：分词 'tallest_'**
```Python
symbols = ['tall_', 'er_', 't', 'a', 'l', ...]

# 尝试1: 'tallest_' ✗ 不在词汇表 → end=7
# 尝试2: 'tallest'  ✗ 不在词汇表 → end=6
# ...
# 尝试6: 'tall'     ✗ 不在词汇表 → end=3
# 尝试7: 'tal'      ✗ 不在词汇表 → end=2
# 尝试8: 'ta'       ✗ 不在词汇表 → end=1
# 尝试9: 't'        ✓ 在词汇表！
cur_output = ['t']
start = 1, end = 8  # 从位置1重新开始
```
**算法示例**

**输入**
```Python
tokens = ['tallest_', 'faster_']
symbols = ['tall_', 'er_', 't', 'a', 'l', 's', 'f', 'e', 'r', '_']
```
**分词过程**

**'tallest_' 的分词**
```python
t a l l e s t _
↑           ↑
start=0    end=8
尝试 'tallest_'? ✗
尝试 'tallest'?  ✗
...
尝试 'tall_'?    ✓ 在词汇表！
输出: ['tall_']
start=4, end=8

    e s t _
    ↑     ↑
尝试 'est_'?    ✗
尝试 'est'?     ✗
...
尝试 'er_'?     ✓ 在词汇表！
输出: ['tall_', 'er_']
start=6, end=8

      t _
      ↑ ↑
尝试 't_'?      ✗
尝试 't'?       ✓ 在词汇表！
输出: ['tall_', 'er_', 't']
start=7, end=8

        _ 
        ↑
尝试 '_'?       ✓ 在词汇表！
输出: ['tall_', 'er_', 't', '_']
start=8 → 完成！

最终结果: 'tall_ er_ t _'
```
**更优的词汇表**

如果 symbols 包含 'er' 和 'est_'：
```Python
'tallest_' →
  1. 'tall_' ✓ （最长匹配）
  2. 'est_' ✓ （剩余部分）
→ 输出: 'tall_ est_'  # 更优分词
```
**算法特点**

**优点**
- 确定性：给定词汇表，分词结果唯一
- 高效：贪心策略时间复杂度 O(n²)，实际优化后更快
- OOV处理：能从未登录词中提取已知子词

**缺点**
- 非最优：贪心可能错过全局最优（但实践中效果良好）
- 词汇表依赖：分词质量取决于 symbols 的覆盖度

In [8]:
def segment_BPE(tokens, symbols):
    # 1. 初始化输出列表：存储所有词元的分割结果
    outputs = []
    '''
    2. 遍历每个词
    token：待分词的字符串（如'tallest_'）
    start：当前子词的起始索引
    end：当前子词的结束索引（尝试长度）
    cur_output：该词的分词结果
    '''
    for token in tokens:
        start, end = 0, len(token)
        cur_output = []
        # 具有符号中可能最长子字的词元段
        # 3. 贪心最长匹配循环
        while start < len(token) and start < end:
            if token[start: end] in symbols:
                cur_output.append(token[start: end])
                start = end
                end = len(token)
            else:
                end -= 1
        # 4. 处理未匹配部分
        # 触发条件：循环结束仍有未匹配的字符；处理：标记为[UNK]（未知词）
        if start < len(token):
            cur_output.append('[UNK]')
        # 5. 拼接结果：将分词列表用空格连接：如['tall_','er_']→'tall_ er_'
        outputs.append(' '.join(cur_output))
    return outputs

我们使用列表`symbols`中的子词（从前面提到的数据集学习）来表示另一个数据集的`tokens`。


In [9]:
tokens = ['tallest_', 'fatter_']
print(segment_BPE(tokens, symbols))

['tall e s t _', 'fa t t er_']
